# Cat vs Non-Cat

Data Format: (examples, features)


### Model: 2 layer Keras
12288  > 60 > 1


* X_train (209, 12288)
* X_test (50, 12288)
* Y_train (209,1)
* Y_test (50,1)

* Train Accuracy -- 99%
* Test Accuracy -- 70-72%


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from lr_utils import load_dataset
import tensorflow as tf
import h5py

from keras.layers import Dense, Activation
from keras.models import Sequential
import keras

C:\Users\abido\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def load_dataset():
    train_dataset = h5py.File('datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes


### Preprocessing

In [4]:
#load dataset
X_train, Y_train, X_test, Y_test, classes = load_dataset()


# Flatten X
X_train = X_train.reshape(X_train.shape[0],-1)
X_test = X_test.reshape(X_test.shape[0],-1)

# Normalize X
X_train = X_train/255.
X_test = X_test/255.

# transpose Y
Y_train = Y_train.T
Y_test = Y_test.T

### Model

In [5]:
# Parameters
learning_rate = 0.005
training_epochs = 500 ## was 2000 originally, was overfitting
batch_size = 100
display_step = 50

In [6]:
def build_model():
    model = Sequential()
    model.add(Dense(60, kernel_initializer='normal', input_dim = 12288, activation = 'relu' ))
    model.add(Dense(1,kernel_initializer='normal', activation = 'sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = build_model()
model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 60)                737340    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 61        
Total params: 737,401
Trainable params: 737,401
Non-trainable params: 0
_________________________________________________________________


In [7]:
## Callback
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 50 == 0: print('')
        print('.', end='')

In [8]:
history = model.fit(X_train, Y_train,callbacks=[PrintDot()],epochs=training_epochs, batch_size= 32, verbose=0)


..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................

#### Evaluate the model

In [9]:
history.history.keys()

dict_keys(['loss', 'acc'])

In [20]:
[loss, accuracy] = model.evaluate(X_test, Y_test,verbose=0)
print("Training Accuracy: % {:.2f}".format(model.evaluate(X_train, Y_train,verbose=0)[1]*100))
print("Test Accuracy: % {:.2f}".format(model.evaluate(X_test, Y_test, verbose=0)[1]*100))

Training Accuracy: % 100.00
Test Accuracy: % 70.00
